In [1]:
from src.data_loader import *
import os

dataset = 0

if dataset:

    path = '../data/AudioMNIST_Indicies/dummy_labels.csv'
    root_dir = '../data/AudioMNIST/'

else:
    path = '../data/AvianID_AcousticIndices/UK_AI.csv'
    root_dir='../data/UK_BIRD/'

# 

ds = AvianNatureSounds(annotation_file=path,root_dir=root_dir,mel_spectrogram=None,mode='stft',key='habitat')
train_loader = DataLoader(dataset=ds, batch_size=16, shuffle=True)
batch, label = next(iter(train_loader)) 

batch.shape

batch[0].shape

torch.Size([2, 513, 93751])

In [2]:
2880000

2880000